<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Coin_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT DATASETS AND LIBRARIES


In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, confusion_matrix, classification_report, accuracy_score, f1_score
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import requests
from requests.exceptions import HTTPError
import json as js
from datetime import datetime, timedelta 
import time
from os.path import exists
from decimal import *
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import KFold, ParameterGrid
from keras.layers import Input, LSTM, Attention, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error


pd.options.display.float_format = '{:f}'.format
np.set_printoptions(formatter={'float': '{:f}'.format})

#Library

In [48]:
# Function to plot interactive plots using Plotly Express
sc = MinMaxScaler()
num_features = 2 #3
candle_features = 5 #6
coin_base = True
ku_coin = False
load_models = True
extra_training = False
COINBASE_REST_API = 'https://api.pro.coinbase.com' 
COINBASE_PRODUCTS = COINBASE_REST_API+'/products'
KUCOIN_REST_API = "https://api.kucoin.com"
KUCOIN_PRODUCTS = KUCOIN_REST_API+ "/api/v1/market/allTickers"
KUCOIN_CANDLES = KUCOIN_REST_API+ "/api/v1/market/candles"

data_path = '/content/drive/My Drive/data/output.csv'
model_path = "/content/drive/My Drive/data"

def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

def get_single_stock(price_df, vol_df, name):
    return pd.DataFrame({'Date': price_df['Date'], 'Close': price_df[name], 'Volume': vol_df[name]})

def scale_data(data):
  # Scale the data
  scaled_data = sc.fit_transform(data)
  return scaled_data

def sort_date(pric_df):
  pric_df = pric_df.sort_values(by = ['Date'])
  return pric_df

def append_price_dif(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  df = df[:-1]
  return df

def append_price_dif_(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  return df

def append_15d_slope(df):
  df['15Close'] = df['Close'].shift(15)
  df['15Date'] = df['Date'].shift(15)
  df['Trend'] = (df['Close'] - df['15Close']) / 15
  df = df[15:]
  return df

def show_plot(data, title):
  plt.figure(figsize = (13, 5))
  plt.plot(data, linewidth = 3)
  plt.title(title)
  plt.grid()

def build_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  outputs = keras.layers.Dense(1, activation='linear')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def build_attention_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  attention_layer = Attention()([x, x])
  outputs = keras.layers.Dense(1, activation='linear')(x)  
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def connect(url, params):   
  response = requests.get(url,params)
  response.raise_for_status()
  return response

def coinbase_json_to_df(delta, product, granularity='86400'):
  start_date = (datetime.today() - timedelta(seconds=delta*int(granularity))).isoformat()
  end_date = datetime.now().isoformat()
  # Please refer to the coinbase documentation on the expected parameters
  params = {'start':start_date, 'end':end_date, 'granularity':granularity}
  response = connect(COINBASE_PRODUCTS+'/' + product + '/candles', params)
  response_text = response.text
  df_history = pd.read_json(response_text)
  # Add column names in line with the Coinbase Pro documentation
  df_history.columns = ['time','low','high','open','close','volume']
  df_history['time'] = [datetime.fromtimestamp(x) for x in df_history['time']]
  return df_history

def ku_coin_json_to_df(delta, product, granularity='86400'):
  granularity = int(granularity)
  start_date = (datetime.today() - timedelta(seconds=delta*granularity))
  end_date = datetime.now()

  # Please refer to the kucoin documentation on the expected parameters
  params = {'startAt':int(start_date.timestamp()), 'endAt':int(end_date.timestamp()), 'type':gran_to_string(granularity), 'symbol':product}
  response = connect(KUCOIN_CANDLES, params)
  response_text = response.text
  response_data = js.loads(response_text);
  if (response_data["code"] != "200000"):
    raise Exception("Illegal response: " + response_text)
  
  df_history = pd.DataFrame(response_data["data"])

  # kucoin is weird in that they don't have candles for everything. IF we don't have the requested
  # number of bars here, it throws off the whole algo. I don't want to try and project so we 
  # just won't trade those instruments
  got_bars = len(df_history)
  if ( got_bars < delta-1):
    raise Exception("Requested:" + str(delta) + " bars " + " but only got:" + str(got_bars))

  df_history.columns = ['time','open','close','high','low','volume', 'amount']
  df_history['time'] = [datetime.fromtimestamp(int(x)) for x in df_history['time']]
  df_history['open'] = [float(x) for x in df_history['open']]
  df_history['close'] = [float(x) for x in df_history['close']]
  df_history['high'] = [float(x) for x in df_history['high']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['volume'] = [float(x) for x in df_history['volume']]
  df_history['amount'] = [float(x) for x in df_history['amount']]
  return df_history
  
def gran_to_string(granularity):
  #todo implement this actually
  return "1day"

#def get_coin_data_frames(time, product, granularity='86400', feature_set = ["Close", "Volume", "Trend"]):
def get_coin_data_frames(time, product, granularity='86400', feature_set = ["Close", "Volume"]):
  if coin_base:
    df_raw = coinbase_json_to_df(time, product, granularity)
  else:
    df_raw = ku_coin_json_to_df(time, product, granularity)

  df_btc_history = df_raw
  if len(df_btc_history.index) == 0:
    print("No data for ", product)
  
  df_btc_history = df_btc_history.rename(columns={"time":"Date", "open":"Open", "high":"High", "low":"Low", "close":"Close", "volume":"Volume"})
  df_btc_history = sort_date(df_btc_history)
  df_btc_history = append_price_dif_(df_btc_history)
  #df_btc_history = append_15d_slope(df_btc_history)
  df_btc_features = df_btc_history[feature_set]
  df_history_scaled = sc.fit_transform(df_btc_features)
  return [df_btc_history, df_btc_features, df_history_scaled, df_raw]

def build_profit_estimate(predicted, df_btc_history):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_btc_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Predicted-Target"] = df_predicted_chart["Predicted"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted-Target"] - df_predicted_chart["Predicted"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["RealDiff"] = df_btc_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_btc_history["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  profit = df_predicted_chart["Profit"].sum()
  return [df_predicted_chart, profit]

def debug_prediction_frame(predicted, df_history, df_history_scaled):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Original"] = df_history_scaled[:,0]
  #Trend
  #df_predicted_chart["Original-Target"] = df_history_scaled[:,2]
  df_predicted_chart["Original-Target"] = df_history_scaled[:,1]
  df_predicted_chart["Target-Date"] = df_predicted_chart["Date"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted"] - df_predicted_chart["Original"]
  df_predicted_chart["Actual-Diff"] = df_predicted_chart["Original-Target"] - df_predicted_chart["Original"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["Close"] = df_history["Close"]
  df_predicted_chart["Target"] = df_history["Target"]
  df_predicted_chart["RealDiff"] = df_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_predicted_chart["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  return df_predicted_chart

def get_all_products():
  if coin_base:
    return get_all_coinbase_products()
  
  if ku_coin:
    return get_all_kucoin_products()

def get_all_kucoin_products():
  response = connect(KUCOIN_PRODUCTS, {})
  products = js.loads(response.text)
  df_products = pd.DataFrame(products["data"]["ticker"])
  df_products = df_products.rename(columns={"symbol":"id"})
  return df_products

def get_all_coinbase_products():
  response = connect(COINBASE_PRODUCTS, {})
  response_text = response.text
  df_products = pd.read_json(response_text)
  return df_products

def predict_trade(model, product, bars, npa_scaled=[]):
  
  if len(npa_scaled) == 0:
    print("downloading...")
    [df_full, df_features, npa_scaled, df_raw] = get_coin_data_frames(bars, product)

  predicted = model.predict(npa_scaled).flatten()
  
  #convert to data frames that have the correct shape for being unscaled
  #df_scaled = pd.DataFrame(npa_scaled, columns = ["Close", "Volume", "Trend"])
  df_scaled = pd.DataFrame(npa_scaled, columns = ["Close", "Volume"])
  
  # I want to believe that scaling happens on a per column basis, we only care about
  # price here so we will dummy out volume and trend and use the scaler on it
  # this kinda sucks, if we add features we'll need to add them here for unscaling
  df_temp = pd.DataFrame(predicted, columns = ["Close"])
  df_temp["Volume"] = 0
  #df_temp["Trend"] = 0
  
  # unscale them both
  #df_temp = pd.DataFrame(sc.inverse_transform(df_temp), columns = ["Close", "Volume", "Trend"])
  #df_trade = pd.DataFrame(sc.inverse_transform(df_scaled), columns = ["Close", "Volume", "Trend"])
  df_temp = pd.DataFrame(sc.inverse_transform(df_temp), columns = ["Close", "Volume"])
  df_trade = pd.DataFrame(sc.inverse_transform(df_scaled), columns = ["Close", "Volume"])
  

  # add predicted
  df_trade["Predicted"] = df_temp["Close"]
  df_trade = df_trade.tail(1)

  # add the product, derive a move and percent
  df_trade["Product"] = row.id;
  df_trade["Move"] = df_trade["Predicted"] - df_trade["Close"]
  df_trade["Percent"] = (df_trade["Move"] / df_trade["Close"]) * 100
  df_trade["RawPercent"] = df_trade["Move"] / df_trade["Close"]
  df_trade["250Fees"] = (250 * 0.004) * 2
  df_trade["5kFees"] = (5000 * 0.004) * 2
  df_trade["10kFees"] = (10000 * 0.0025) * 2
  df_trade["250Profit"] = (250 * df_trade["RawPercent"]) - df_trade["250Fees"] 
  df_trade["5kProfit"] = (5000 * df_trade["RawPercent"]) - df_trade["5kFees"]
  df_trade["10k0Profit"] = (10000 * df_trade["RawPercent"]) - df_trade["10kFees"]
  return df_trade

def get_yf_training_set_for(df):
  target_df = append_price_dif(df)
  #target_df = append_15d_slope(target_df)
  #features = target_df[["Open", "High", "Low", "Close", "Volume", "Trend", "Target"]]
  features = target_df[["Open", "High", "Low", "Close", "Volume", "Target"]]
  scaled_features = scale_data(features)
  return extract_training(scaled_features, len(target_df),len(features.columns)-1)

def extract_training(scaled_features, length, num_features):
  X = []
  y = []
  
  for i in range(0, length):
    X.append(scaled_features [i][0:num_features])
    y.append(scaled_features [i][num_features])
  X = np.asarray(X)
  y = np.asarray(y)
  return [scaled_features, X, y]

def get_training_set_for(ticker):
  target_df = get_single_stock(all_stocks_price_df, all_stocks_vol_df, ticker)
  target_df = append_price_dif(target_df)
  #target_df = append_15d_slope(target_df)
  #features = target_df[["Close", "Volume", "Trend", "Target"]]
  features = target_df[["Close", "Volume", "Target"]]
  scaled_features = scale_data(features)
  return extract_training(scaled_features, len(target_df), len(features.columns)-1)
  
def train_model(model, X, y):

  # One day we might need test, but for now we don't we can use another
  # time series, we have so many
  # Split the data
  #split = int(0.7 * len(X))
  #X_train = X[:split]
  #y_train = y[:split]
  #X_test = X[split:]
  #y_test = y[split:]

  # Reshape the 1D arrays to 3D arrays to feed in the model
  X_train = np.reshape(X, (X.shape[0], X.shape[1], 1))
  
  # Create an early stopping callback
  early_stopping = EarlyStopping(monitor='val_loss', patience=5)
  
  history = model.fit(
      X_train, y,
      epochs = 20,
      batch_size = 32,
      validation_split = 0.2,
      callbacks=[early_stopping]
  )
  return [model, history]

def get_group_bars(df):
  df = pd.DataFrame(sc.fit_transform(df[["Close", "Volume"]]), columns=["Close","Volume"])
  # Split into input sequences and target values
  n_steps = 4*4  # 4 hours of data at 15 minute intervals
  X = []
  Y = []
  for i in range(0, len(df), n_steps):
    df_group = df.iloc[i:i+n_steps]
    if len(df_group) != n_steps:
      continue
    X.append(np.array(df_group.values))
    Y.append(df_group.values[-1,0])
    
  # Convert the lists to NumPy arrays
  X = np.array(X)
  Y = np.array(Y)
  return [X, Y]

def build_15m_model():
  # the 15 min bar model
  # Build the model
  group_size = 4*4
  features = 2
  model15 = build_model(group_size, features)
  
  # Compile the model
  model15.compile(loss='mean_squared_error', optimizer='adam')
  
  # Train it
  tickers = ["SPY", "IBM", "TSLA", "CAT", "XOM", "B", "F", "AAPL", "AMZN"]
  data_path = '/content/drive/My Drive/data'
  early_stopping = EarlyStopping(monitor='val_loss', patience=5)
  for ticker in tickers:
    file_path = data_path + "/" + ticker + "-15.csv"
    df = sort_date(pd.read_csv(file_path).rename(columns={"Datetime":"Date"}))
    df['Date'] = pd.to_datetime(df['Date'])
    [X,Y] = get_group_bars(df)
    model15.fit(X, Y, 
      epochs = 20,
      batch_size = 32,
      validation_split = 0.2,callbacks=[early_stopping])
  return model15

#pull training data 
all_stocks_price_df = sort_date(pd.read_csv('/content/drive/My Drive/data/stock.csv'))
all_stocks_vol_df = sort_date(pd.read_csv("/content/drive/My Drive/data/stock_volume.csv"))
spy_df = sort_date(pd.read_csv('/content/drive/My Drive/data/SPY.csv'))
cat_df = sort_date(pd.read_csv('/content/drive/My Drive/data/CAT.csv'))
f_df = sort_date(pd.read_csv('/content/drive/My Drive/data/F.csv'))
xom_df = sort_date(pd.read_csv('/content/drive/My Drive/data/XOM.csv'))
ibm_df = sort_date(pd.read_csv('/content/drive/My Drive/data/IBM.csv'))
spy_15_df = sort_date(pd.read_csv('/content/drive/My Drive/data/SPY-15.csv').rename(columns={"Datetime":"Date"}))

# Get or Train a Model

In [6]:
if load_models:
  #load models
  print("loading models from disk")
  model_orig = keras.models.load_model(model_path + "/model_orig.h15")
  model_ohlc = keras.models.load_model(model_path + "/model_ohlc.h15")
  model_att1 = keras.models.load_model(model_path + "/model_att1.h15")
  model_att2 = keras.models.load_model(model_path + "/model_att2.h15")
  model_15m  = keras.models.load_model(model_path + "/model_15m.h15")
else:
  #if file_exists:
  #  print("hello")
  #  model = keras.models.load_model(model_path)
  #else:

  [scaled_features, X, y] = get_training_set_for("sp500")  
  [scaled_features1, X1, y1] = get_yf_training_set_for(spy_df)  

  model_orig = build_model(num_features, 1)
  model_ohlc = build_model(candle_features, 1)
  model_att1 = build_attention_model(num_features, 1)
  model_att2 = build_attention_model(candle_features, 1)

  [model_att1, history] = train_model(model_att1, X, y)
  [model_orig, history] = train_model(model_orig, X, y)
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)
  [model_att2, history] = train_model(model_att2, X1, y1)
  model_15m = build_15m_model()

  model_orig.save(model_path + "/model_orig.h15")
  model_ohlc.save(model_path + "/model_ohlc.h15")
  model_att1.save(model_path + "/model_att1.h15")
  model_att2.save(model_path + "/model_att2.h15")
  model_15m.save(model_path + "/model_15m.h15")


loading models from disk


In [ ]:
if extra_training:
  [scaled_features, X, y] = get_training_set_for("IBM")  
  [model_orig, history] = train_model(model_orig, X, y)
  [scaled_features, X, y] = get_training_set_for("T")  
  [model_orig, history] = train_model(model_orig, X, y)
  [scaled_features, X, y] = get_training_set_for("BA")  
  [model_orig, history] = train_model(model_orig, X, y)
  [scaled_features, X, y] = get_training_set_for("TSLA")  
  [model_orig, history] = train_model(model_orig, X, y)


In [ ]:
# additional training?
if extra_training:

  [scaled_features1, X1, y1] = get_yf_training_set_for(cat_df)  
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)
  [scaled_features1, X1, y1] = get_yf_training_set_for(f_df)  
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)
  [scaled_features1, X1, y1] = get_yf_training_set_for(ibm_df)  
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)
  [scaled_features1, X1, y1] = get_yf_training_set_for(xom_df)  
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)

# Visualize and Backtest

## backtest the main models

In [7]:

# Run Random Search flat, attention flat, random ohlc and flat ohlc against these two models
# Loss isn't cutting it, its always 0?
# After we find the winner above, run it trained on spy vs trained on all

[btc_history, df_btc_features, df_history_scaled, df_raw] = get_coin_data_frames(180, "ETH-USDT")
#[btc_history1, df_btc_features1, df_history_scaled1, df_raw1] = get_coin_data_frames(180, "FCON-USDT", 86400, ["Open", "High", "Low", "Close", "Volume", "Trend"])
[btc_history1, df_btc_features1, df_history_scaled1, df_raw1] = get_coin_data_frames(180, "ETH-USDT", 86400, ["Open", "High", "Low", "Close", "Volume"])

budget = 3000

scaled_features = scale_data(btc_history[["Close"]])
scaled_features1 = scale_data(btc_history1[["Close"]])

predicted_orig = model_orig.predict(df_history_scaled).flatten()
predicted_ohlc = model_ohlc.predict(df_history_scaled1).flatten()
predicted_att1 = model_att1.predict(df_history_scaled).flatten()
predicted_att2 = model_att2.predict(df_history_scaled1).flatten()

print("predicted_orig mse: ", mean_squared_error(scaled_features, predicted_orig))
print("predicted_ohlc mse: ", mean_squared_error(scaled_features1, predicted_ohlc))
print("predicted_att1 mse: ", mean_squared_error(scaled_features, predicted_att1))
print("predicted_att2 mse: ", mean_squared_error(scaled_features1, predicted_att2))

print("predicted_orig mae: ", mean_absolute_error(scaled_features, predicted_orig))
print("predicted_ohlc mae: ", mean_absolute_error(scaled_features1, predicted_ohlc))
print("predicted_att1 mae: ", mean_absolute_error(scaled_features, predicted_att1))
print("predicted_att2 mae: ", mean_absolute_error(scaled_features1, predicted_att2))


[df_profit, profit1] = build_profit_estimate(predicted_orig, btc_history)
[df_profit, profit2] = build_profit_estimate(predicted_ohlc, btc_history1)
[df_profit, profit3] = build_profit_estimate(predicted_att1, btc_history)
[df_profit, profit4] = build_profit_estimate(predicted_att2, btc_history1)


df_chart = debug_prediction_frame(predicted_orig, btc_history, df_history_scaled)
df_chart["Predicted-ohlc"] = predicted_ohlc
df_chart["Predicted-att1"] = predicted_att1
df_chart["Predicted-att2"] = predicted_att2

#interactive_plot(df_chart[["Date","Original", "Predicted-ohlc", "Predicted", "Predicted-atten", "Predicted-opt"]], "Wtf")
interactive_plot(df_chart[["Date","Original", "Predicted-ohlc", "Predicted", "Predicted-att1", "Predicted-att2"]], "Wtf")
print("Profits:", profit1, profit2, profit3, profit4)




6/6 [==============================] - 1s 22ms/step
predicted_orig mse:  0.0007851460039768313
predicted_ohlc mse:  0.0015883648815974486
predicted_att1 mse:  0.0007909360242010586
predicted_att2 mse:  0.002483008703922198
predicted_orig mae:  0.020866257338089302
predicted_ohlc mae:  0.02779270813090434
predicted_att1 mae:  0.023012427605687193
predicted_att2 mae:  0.03419388481266834


Profits: 7888.455457637146 4737.70206871736 7503.117363391886 2754.5922231649033


## backtest the 15m model

In [49]:
model_15m = build_15m_model()

Epoch 1/20
2/2 [==============================] - 9s 2s/step - loss: 0.3459 - val_loss: 0.0106
Epoch 2/20
2/2 [==============================] - 1s 279ms/step - loss: 0.0493 - val_loss: 0.0646
Epoch 3/20
2/2 [==============================] - 0s 240ms/step - loss: 0.0997 - val_loss: 0.0049
Epoch 4/20
2/2 [==============================] - 1s 301ms/step - loss: 0.0151 - val_loss: 0.0068
Epoch 5/20
2/2 [==============================] - 0s 220ms/step - loss: 0.0578 - val_loss: 0.0056
Epoch 6/20
2/2 [==============================] - 0s 231ms/step - loss: 0.0440 - val_loss: 0.0032
Epoch 7/20
2/2 [==============================] - 0s 137ms/step - loss: 0.0129 - val_loss: 0.0166
Epoch 8/20
2/2 [==============================] - 0s 128ms/step - loss: 0.0181 - val_loss: 0.0269
Epoch 9/20
2/2 [==============================] - 0s 137ms/step - loss: 0.0257 - val_loss: 0.0134
Epoch 10/20
2/2 [==============================] - 0s 164ms/step - loss: 0.0106 - val_loss: 0.0046
Epoch 11/20
2/2 [=====

In [50]:
df_raw = coinbase_json_to_df(180, "BTC-USD", 900)
df_raw = df_raw.rename(columns={"close":"Close", "time":"Date", "volume":"Volume"})
df_raw = df_raw[["Date", "Close", "Volume"]]
[X,Y] = get_group_bars(df_raw)

predicted = model_15m.predict(X).flatten()
df = pd.DataFrame()
df["Date"] = df_raw["Date"]
df["Close"] = sc.fit_transform(df_raw[["Close"]])
n_steps = 16
expanded_prd = []
last = len(predicted) -1
for i in range(0, len(df)):
  pos = int(i/n_steps)
  if pos > last:
    expanded_prd.append(predicted[-1])
  else:
    expanded_prd.append(predicted[pos])
df["Predicted"] = expanded_prd
interactive_plot(df[["Date", "Close", "Predicted"]], "Wtf")

1/1 [==============================] - 1s 1s/step


In [51]:
model_15m.save(model_path + "/model_15m.h15")

# What has a buy indicator for tomorrow?

In [ ]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]

if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
bars = 91
counter = 0;
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    [df_full, df_features, npa_scaled, df_raw] = get_coin_data_frames(bars, row.id)
    df_trade = predict_trade(model_orig, row.id, bars, npa_scaled)
    df_trade_ohlc = predict_trade(model_ohlc, row.id, bars, npa_scaled)
    df_trade_att1 = predict_trade(model_att1, row.id, bars, npa_scaled)
    df_trade_att2 = predict_trade(model_att2, row.id, bars, npa_scaled)
    
    # we need to unscale the predicted values so that we have an entry and exit point
    # entry should be roughly close and exit should be roughly predicted

    # Stick this on the end of the main dataframe
    df_trade["prd-ohlc"] = df_trade_ohlc["Predicted"]
    df_trade["pct-ohlc"] = df_trade_ohlc["Percent"]
    df_trade["prd-att1"] = df_trade_att1["Predicted"]
    df_trade["pct-att1"] = df_trade_att1["Percent"]
    df_trade["prd-att2"] = df_trade_att2["Predicted"]
    df_trade["pct-att2"] = df_trade_att2["Percent"]
    df_trades = df_trades.append(df_trade);
    
    #counter+=1
    #if counter > 5:
    #  break
  except Exception as inst:
    #raise inst
    print("Error: ", inst)
  time.sleep(1)
df_trades.reset_index()
df_buys = df_trades[df_trades['Move'] > 0] 
df_shorts = df_trades[df_trades['Move'] < 0] 




In [ ]:
df_shorts


In [ ]:
df_trade = predict_trade(model_orig, "REN-USD", 90)
df_trade

In [ ]:
with open(data_path, 'w', encoding = 'utf-8-sig') as f:
  df_trades.to_csv(f)

In [ ]:
df_shorts[df_shorts["Product"] == "MPL-USD"]